In [1]:
import numpy as np
import tensorflow as tf
from lc import analysis, graph, config, loader
from tensorflow.contrib.layers import fully_connected

/home/kaidong/anaconda3/envs/tf/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
np.set_printoptions(precision=2, suppress=True)

In [3]:
def five_layers(x, ref_y):
    activation = graph.max_out
    for i in range(6):
        x = fully_connected(
            x,
            1000,
            activation_fn=activation,
            scope="layer" + str(i))
    y = fully_connected(
        x, 1, activation_fn=tf.identity, scope="fc")
    diff = tf.reduce_sum(y) * -1
    tf.add_to_collection("losses", diff)
    tf.add_to_collection("ptime", y)

    
def gradClips(shape, minEdge, maxEdge, length):
    x = tf.Variable(
        tf.add(tf.multiply(
            tf.random_uniform((length, shape)), (maxEdge-minEdge)),
            minEdge), expected_shape=(length, shape)
    )
    tf.add_to_collection("tVar", x)
    maxVal = np.tile(maxEdge, length).reshape((length, shape))
    minVal = np.tile(minEdge, length).reshape((length, shape))
    
    return tf.clip_by_value(x, minVal, maxVal)

In [4]:
config.DATAFILE = "time_avg"
data = loader.Loader({"name": "GDopt", "discription": ""})

config.NUM_UNIT = 10
minEdge = np.round(np.min(data.X, 0), 1)
maxEdge = np.round(np.max(data.X, 0), 1)

In [5]:
batchSize = 256
gradGraph = tf.Graph()
with gradGraph.as_default():
    five_layers(
        gradClips(31, minEdge, maxEdge, batchSize), np.ones([batchSize])*200
    )
    fin_loss = analysis.fin_loss()
    trainVar, = gradGraph.get_collection("tVar")
    pred, = gradGraph.get_collection("ptime")
    optimizer = tf.train.AdamOptimizer()
    opt = optimizer.minimize(fin_loss, var_list=[trainVar])
    init = tf.global_variables_initializer()
    model = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)[1:]

In [6]:
sess = tf.Session(graph=gradGraph)

with gradGraph.as_default():
    sess.run(init)    
    saver = tf.train.Saver(model)
    saver.restore(sess, tf.train.latest_checkpoint("saved"))
    
gradGraph.finalize()

INFO:tensorflow:Restoring parameters from saved/chkpnt-8000


In [7]:
timelist = []
paralist = []
for i in range(50000):
    _, predVal = sess.run([opt, pred])
    if not (i % 100):
        timelist.append(predVal.reshape(-1))
        paralist.append(sess.run(trainVar))

In [8]:
#paralist, timelist = load("./datamax")

In [8]:
t = np.array(timelist)

In [9]:
from matplotlib import pyplot as plt
%matplotlib 
plt.style.use('ggplot')

Using matplotlib backend: Qt5Agg


In [10]:
plt.figure()

In [11]:
plt.plot(t)

In [12]:
from xilio import dump, load
import numpy as np

In [13]:
dump("datamaxmass", (paralist, timelist))

In [15]:
ft = timelist[-1]

In [16]:
np.argwhere(ft == max(ft))

array([[26]])

In [17]:
max(ft)

112.06644

In [22]:
print(np.round(paralist[-1][26],2))

[-15.01  -4.99  12.51  44.99  15.01  11.42  88.01  31.99  28.01 -42.51
 -77.51 -36.35   2.51  59.01 -17.49  40.98 -57.49  39.53   9.99 -22.98
 -19.02  29.49  55.49   5.    82.01  -3.51  24.52 -45.01   9.01  43.53
 -15.01]
